[View in Colaboratory](https://colab.research.google.com/github/ts-kim/models/blob/master/structured_self_attention.ipynb)

##참고문헌
**Zhouhan Lin, et al.**, *A Structured Self-attentive Sentence Embedding*

https://arxiv.org/pdf/1703.03130.pdf</br>
https://arxiv.org/abs/1703.03130


https://simonjisu.github.io/datascience/2018/04/03/nsmcbidreclstmselfattn.html

In [1]:
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision

import torch
print(torch.__version__)
print(torch.cuda.is_available())

# pytorch 0.4.0 설치

!pip install pytorch-nlp
# torchnlp 설치(데이터셋 다운로드용)

!pip install nltk
!pip install gensim
# Word2Vec 설치(워드 임베딩용)

tcmalloc: large alloc 1073750016 bytes == 0x5bb40000 @  0x7f7296ded1c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
0.4.0
True
    100% |████████████████████████████████| 102kB 7.3MB/s 
    100% |████████████████████████████████| 51kB 17.1MB/s 
    100% |████████████████████████████████| 194kB 20.7MB/s 
  Running setup.py bdist_wheel for ujson ... - \ | / - \ | done
  Stored in directory: /content/.cache/pip/wheels/28/77/e4/0311145b9c2e2f01470e744855131f9e34d6919687550f87d1
Successfully built ujson
    100% |████████████████████████████████| 23.5MB 1.6MB/s 
    100% |████████████████████████████████| 1.4MB 9.5MB/s 
    100% |████████████████████████████████| 133kB 22.0MB/s 
    100% |████████████████████████████████| 61kB 18.0MB/s 
    100% |█████████

In [0]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

import pandas as pd
import numpy as np


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

##사용한 데이터

The Stanford Natural Language Inference (SNLI) Corpus
https://nlp.stanford.edu/projects/snli/

In [3]:
from torchnlp.datasets import snli_dataset
train = snli_dataset(train=True)
test = snli_dataset(test=True)


snli_1.0.zip: 94.6MB [00:09, 10.3MB/s]                            


In [4]:
'''print(test[0]['premise'])
print(test[0]['label'])
print(test[0]['hypothesis'])

print()
print(test[1]['premise'])
print(test[1]['label'])
print(test[1]['hypothesis'])

print()
print(test[2]['premise'])
print(test[2]['label'])
print(test[2]['hypothesis'])'''

"print(test[0]['premise'])\nprint(test[0]['label'])\nprint(test[0]['hypothesis'])\n\nprint()\nprint(test[1]['premise'])\nprint(test[1]['label'])\nprint(test[1]['hypothesis'])\n\nprint()\nprint(test[2]['premise'])\nprint(test[2]['label'])\nprint(test[2]['hypothesis'])"

In [0]:
#데이터 불필요한 부분 제거
train_ls = list(train)
test_ls = list(test)

pd_train = pd.DataFrame(train_ls)
pd_test = pd.DataFrame(test_ls)

del pd_train['hypothesis_transitions']
del pd_train['premise_transitions']
del pd_train['premise']

del pd_test['hypothesis_transitions']
del pd_test['premise_transitions']
del pd_test['premise']



### text to vecs

In [6]:
import nltk
import gensim
from gensim.models.word2vec import Word2Vec
from nltk import word_tokenize

nltk.download('punkt') # this is for word_tokenizing

train_sentences =[]
test_sentences = []

for i in range(len(train)):
  train_sentences.append(train[i]['hypothesis'])
  train_sentences[i]=word_tokenize(train_sentences[i])
  
for i in range(len(test)):
  test_sentences.append(test[i]['hypothesis'])
  test_sentences[i]=word_tokenize(test_sentences[i])

sentences = train_sentences + test_sentences

[nltk_data] Downloading package punkt to /content/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
# word2vec 설정
d_dim = 100


#word2vec 학습
%time embedding_model = Word2Vec(sentences, size=d_dim, iter= 20, min_count = 0)
embedding_model.save("word2vec.model")

CPU times: user 3min 7s, sys: 1.08 s, total: 3min 9s
Wall time: 1min 40s


In [8]:
embedding_model = Word2Vec.load("word2vec.model")
print(embedding_model.most_similar(positive=["house"], topn=5))
#0.6 < mansion, apartment

[('mansion', 0.7233067154884338), ('apartment', 0.6885494589805603), ('cabin', 0.6371487975120544), ('workplace', 0.6230380535125732), ('office', 0.6145042181015015)]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [9]:
'''print(embedding_model.wv[sentences[30]].shape) 
print('n =11(# of words in sentence), d=100(dim of word2vec) ( S = n by d dim)')'''

"print(embedding_model.wv[sentences[30]].shape) \nprint('n =11(# of words in sentence), d=100(dim of word2vec) ( S = n by d dim)')"

In [10]:
'''#최대길이는? --- just for checking
max_len=0
for i in range(len(sentences)):
    if len(sentences[i])>max_len:
        max_len=len(sentences[i])
print(max_len)'''

'#최대길이는? --- just for checking\nmax_len=0\nfor i in range(len(sentences)):\n    if len(sentences[i])>max_len:\n        max_len=len(sentences[i])\nprint(max_len)'

In [0]:
def padding(x,length): # 넘파이 어레이를 적당한 길이로 자르거나 늘려줌
    _ = length-len(x)
    if(_>0):
        a = np.zeros((_,100))
        x = np.concatenate((x,a))
    if(_<0):
        x = x[:length,:]
    return x

In [12]:
length = 9
X_train = []
X_test = []
for i in range(len(train_sentences)):
  X_train.append(padding(embedding_model.wv[train_sentences[i]],length))
  if i%100000 ==99999 :
    print(i+1,X_train[i].shape)

for i in range(len(test_sentences)):
  X_test.append(padding(embedding_model.wv[test_sentences[i]],length))
print(len(test_sentences))
print(X_test[0].shape)

100000 (9, 100)
200000 (9, 100)
300000 (9, 100)
400000 (9, 100)
500000 (9, 100)
10000
(9, 100)


In [13]:
'''X_test[2].shape'''

'X_test[2].shape'

In [0]:
### y_data 준비


from sklearn import preprocessing

save_label = []
le = preprocessing.LabelEncoder()

#y_train to index
le.fit(pd_train['label'])
save_label.append(le.classes_)
pd_train['label'] = le.transform(pd_train['label'])

#y_test to index
le.fit(pd_test['label'])
save_label.append(le.classes_)
pd_test['label'] = le.transform(pd_test['label'])


y_train = pd_train['label']
y_test = pd_test['label']

In [0]:
X_train = torch.tensor(X_train).type(torch.FloatTensor).to(device)
y_train = torch.tensor(y_train.values).type(torch.LongTensor).to(device)

X_test = torch.tensor(X_test).type(torch.FloatTensor).to(device)
y_test = torch.tensor(y_test.values).type(torch.LongTensor).to(device)


In [0]:
train_dataset = torch.utils.data.TensorDataset(X_train, y_train) # X_train : float tensor, y_train : long tensor
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)


# model  

In [0]:
#hyper parameters
input_size = 100 # 임베딩 사이즈 (임베딩시에 변동)
hidden_size = 400 # 히든 사이즈 u  (모델에서 변동)
attention_dim = 300 #da, (모델에서 변동)
num_layers = 1 # 몇층의 LSTM? (현재 참고논문 모델은 1)
r = 1 # number of attentions

batch_size = 20
epochs = 3
learning_rate = 0.00005

#lstm input : (batch#, seq_len, embedding_size)
#lstm 정의할때 : input_size = embedding_size , hidden_size = hidden_size, num_layers = 몇층?

In [0]:
class SelfAttention(nn.Module): 
  # input_size = embedding_size , hidden_size = sequence_length , num_layers = embedding_dimension
    def __init__(self, input_size, hidden_size, num_layers, attention_dim, batch_size, r):
        super(SelfAttention, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.batch_size = batch_size
        self.lstm = nn.LSTM(input_size = input_size ,hidden_size = hidden_size ,\
                            num_layers = num_layers, batch_first=True, bidirectional=True) #
        
        self.attention1 = nn.Parameter(torch.randn(attention_dim,2*hidden_size)) # da by 2u W1
        self.attention2 = nn.Parameter(torch.randn(r,attention_dim)) # r by da W2
        self.tanh = nn.Tanh()

        self.softmax = nn.Softmax() 
        
        self.linear = nn.Linear(r*2*hidden_size,4)
        
        
        
    def init_LSTM(self):
        h0 = Variable(torch.zeros(self.num_layers*2, self.batch_size , self.hidden_size)).to(device)
        c0 = Variable(torch.zeros(self.num_layers*2, self.batch_size , self.hidden_size)).to(device)
        return h0, c0
        
    def forward(self, x):
        h0, c0 = self.init_LSTM()
        h, _ = self.lstm(x, (h0, c0))
        
        att = torch.matmul(self.attention1,torch.transpose(h,1,2)) # torch.mm, -> 2d dot 2d / 3d dot 2d -> matmul / bmm 3d dot 3d
        att = self.tanh(att)
        att = torch.matmul(self.attention2, att)
        att = self.softmax(att) # batchsize by r by n
                                # batchsize by n by 2u
        out = torch.bmm(att,h)
        out = out.reshape(batch_size,-1)
        out = self.linear(out)
        return out

In [0]:
model = SelfAttention(input_size, hidden_size, num_layers,attention_dim, batch_size, r).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, drop_last=True) 
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, drop_last= True)

#iteration

In [20]:
for epoch in range(epochs):
    epoch_loss = 0
    batch_loss = 0
    i = 0
    for step, (X_batch, y_batch) in enumerate(train_loader):
        i+=1
        optimizer.zero_grad()

        y_pred = model(X_batch)
        loss = criterion(y_pred,y_batch)
        loss.backward()
        optimizer.step()
        
        _loss = loss.data[0]
        epoch_loss += _loss
        batch_loss += _loss
        if i % 1000 == 999 : 
            print('num_iter =',i+1,'/',len(train_loader), '     loss = ',batch_loss.item()/1000)
            batch_loss = 0
    print('epoch =',epoch+1,'     epoch_loss =',epoch_loss.item()/len(train_loader) )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  


num_iter = 1000 / 27507      loss =  1.2830445556640624
num_iter = 2000 / 27507      loss =  1.201669921875
num_iter = 3000 / 27507      loss =  1.16010986328125
num_iter = 4000 / 27507      loss =  1.116328369140625
num_iter = 5000 / 27507      loss =  1.080173095703125
num_iter = 6000 / 27507      loss =  1.0642777099609375
num_iter = 7000 / 27507      loss =  1.0456845703125
num_iter = 8000 / 27507      loss =  1.035610107421875
num_iter = 9000 / 27507      loss =  1.0186367797851563
num_iter = 10000 / 27507      loss =  1.0190184326171876
num_iter = 11000 / 27507      loss =  1.0194674682617189
num_iter = 12000 / 27507      loss =  1.0029635620117188
num_iter = 13000 / 27507      loss =  0.99550634765625
num_iter = 14000 / 27507      loss =  1.0018256225585938
num_iter = 15000 / 27507      loss =  0.9854664916992187
num_iter = 16000 / 27507      loss =  0.9774651489257813
num_iter = 17000 / 27507      loss =  0.97765380859375
num_iter = 18000 / 27507      loss =  0.9689048461914063

#성능 

56.63 epochs = 20  lr = 5e-05  batch size = 20  hidden size = 200 da = 30


In [21]:
### 10. test set accuracy
model.eval() #평가 모드
with torch.no_grad():
    correct = 0
    total = 0
    for (inputs, labels) in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    test_acc = 100*correct/total
print(test_acc,'epochs =',epochs, ' lr =',learning_rate,' batch size =',batch_size, ' hidden size =',hidden_size, 'da =',attention_dim)
model.train()# 평가 모드 원상복구

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


61.43 epochs = 3  lr = 5e-05  batch size = 20  hidden size = 400 da = 300


SelfAttention(
  (lstm): LSTM(100, 400, batch_first=True, bidirectional=True)
  (tanh): Tanh()
  (softmax): Softmax()
  (linear): Linear(in_features=800, out_features=4, bias=True)
)